In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

In [2]:
df = pd.read_csv('../input/sibur20-naming-data/train.csv')

In [3]:
df.head()

,pair_id,name_1,name_2,is_duplicate
0,1,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0
1,2,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0
2,3,"Rishichem Distributors Pvt., Ltd.",Dsa,0
3,4,Powermax Rubber Factory,Co. One,0
4,5,Tress A/S,Longyou Industries Park Zhejiang,0


In [4]:
len(df)

497819

In [5]:
df_1 = df.loc[df['is_duplicate'] == 1]
df_0 = df.loc[df['is_duplicate'] == 0]

In [6]:
!pip install transliterate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.2 MB/s eta 0:00:00


In [7]:
from transliterate import translit, get_available_language_codes

In [8]:
from itertools import chain

def make_polylang_data(df, label = 0):
    df = df[['name_1','name_2']].values.tolist()
    new = []
    for lang in get_available_language_codes():
        prob = []
        for i in range(len(df)):
            fword = df[i][0]
            sword = df[i][1]
            fnword = translit(fword, lang)
            snword = translit(sword, lang)
            if label == 1:
                prob.append([fword, fnword])
                prob.append([sword, snword])
            else:
                prob.append([fword, snword])
                prob.append([sword, fnword])
        new.append(prob)
    some = []
    for l in range(len(new)):
        prob = []
        for i in range(len(new[l])):
            word = new[l][i][1]
            nword = new[l][i][0]
            prob.append([word, nword])
        some.append(prob)
    new_df = new + some
    new_df = list(chain(*new_df))
    for t in new_df:
        t.append(label)
    return new_df
    

In [9]:
from random import shuffle

In [10]:
def filter_data(df, stop):
    return df.loc[(df.name_1.str.len() > 15)&(df.name_2.str.len() > 15)][:stop]

def get_data(df):
    df_0 = df.loc[df['is_duplicate'] == 0]
    df_1 = df.loc[df['is_duplicate'] == 1]
    if len(df_0) > len(df_1):
        df_0 = filter_data(df_0, len(df_1))
    else:
        df_1 = filter_data(df_1, len(df_0))
    data1 = make_polylang_data(df_1, label = 1)
    data2 = make_polylang_data(df_0, label = 0)
    data = data1 + data2
    shuffle(data)
    return data

In [11]:
data = get_data(df)

In [12]:
dff = pd.DataFrame(columns=['name_1', 'name_2', 'is_dup'], data=data)

In [13]:
dff.head()

,name_1,name_2,is_dup
0,"Pu Max Trading (T) Co., Ltd.","Т.А. Цорпоратион Пвт., Лтд.",0
1,Схалфин Традинг Цо. Wлл,Dhaka Transport And Trading.,0
2,SOPREMA GmbH,СОПРЕМА ГмбХ,1
3,Kuraray India Private Ltd.,Spiro Hoses India Private Ltd.,0
4,Вибрацоустиц Индиа Привате Лтд.,Wictor Compounding (India) Private Ltd.,0


In [14]:
dff.to_csv('data.csv', index = False)